In [1]:
import dlib
import cv2
import numpy as np
from image_processing import detect_pupil, convert_eye_to_binary, extract_eye_region
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [2]:
def pre_process_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dlib_faces = detector(gray)


    processed_data = []

    for dlib_face in dlib_faces:
        shape = predictor(gray, dlib_face)

        for (i, (start, end)) in enumerate([(36,42), (42,48)]):
            eye_image, (eye_min_x, eye_min_y, eye_max_x, eye_max_y) = extract_eye_region(gray, shape, range(start, end))
            eye_image_b = convert_eye_to_binary(eye_image)
            pupil_center, _ = detect_pupil(eye_image_b)

            if pupil_center:
                pupil_center_global = (pupil_center[0] + eye_min_x, pupil_center[1] + eye_min_y)
                pupil_center_global = tuple(pc.item() if isinstance(pc, np.generic) else pc for pc in pupil_center_global)
                bounding_box = (eye_min_x, eye_min_y, eye_max_x - eye_min_x, eye_max_y - eye_min_y)
                bounding_box = tuple(bb.item() if isinstance(bb, np.generic) else bb for bb in bounding_box)
                eye_data = {
                    'eye_position': 'left' if i == 0 else 'right',
                    'pupil_center': pupil_center_global,
                    'bounding_box': bounding_box
                }
                processed_data.append(eye_data)

                left_eye_info = None
                right_eye_info = None
                left_eye_bbox = None
                right_eye_bbox = None

                for eye_data in processed_data:
                    if eye_data['eye_position'] == 'left':
                        left_eye_info = eye_data['pupil_center']
                        left_eye_bbox = eye_data['bounding_box']
                    else:
                        right_eye_info = eye_data['pupil_center']
                        right_eye_bbox = eye_data['bounding_box']
        break
    return processed_data, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, shape

In [3]:
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt

def plot_data_on_image(image, left_eye_box, right_eye_box, left_pupil, right_pupil, output_path):
    fig, ax = plt.subplots()
    ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

    # Draw the eye boxes and pupils
    ax.add_patch(plt.Rectangle((left_eye_box[0], left_eye_box[1]), left_eye_box[2], left_eye_box[3], fill=False, edgecolor='green', linewidth=2))
    ax.add_patch(plt.Rectangle((right_eye_box[0], right_eye_box[1]), right_eye_box[2], right_eye_box[3], fill=False, edgecolor='green', linewidth=2))
    ax.plot(left_pupil[0], left_pupil[1], 'ro', markersize=1)
    ax.plot(right_pupil[0], right_pupil[1], 'ro', markersize=1)

    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()
    print(f"Processed image saved to {output_path}")

def test_eye_gaze_detection(image_path):
    image = cv2.imread(image_path)
    processed_data, _, _, _, _, _ = pre_process_image(image)
    # Process the data
    for data in processed_data:
        if data['eye_position'] == 'left':
            left_eye_box = data['bounding_box']
            left_pupil = data['pupil_center']
        elif data['eye_position'] == 'right':
            right_eye_box = data['bounding_box']
            right_pupil = data['pupil_center']

    # Ensure that data for both eyes is available before plotting
    if left_eye_box is not None and right_eye_box is not None:
        output_path = f"./processed_{os.path.basename(image_path)}"
        plot_data_on_image(image, left_eye_box, right_eye_box, left_pupil, right_pupil, output_path)


# Specify the subdirectory containing the test images
subdirectory = './data/eloise/calibration_images'

# List all images in the subdirectory and process each image
test_image_paths = [os.path.join(subdirectory, f) for f in os.listdir(subdirectory) if f.endswith('.jpg') or f.endswith('.png')]
for image_path in test_image_paths:
    test_eye_gaze_detection(image_path)


Detecting pupil
Contours:  (array([[[ 0,  0]],

       [[ 0,  6]],

       [[ 2,  6]],

       [[ 2,  2]],

       [[ 3,  1]],

       [[ 7,  1]],

       [[ 9,  3]],

       [[ 9,  6]],

       [[17,  6]],

       [[17,  3]],

       [[18,  2]],

       [[22,  2]],

       [[23,  3]],

       [[23,  6]],

       [[24,  6]],

       [[24,  0]]], dtype=int32),)
Contour area:  88.5
Approx area:  107.5
Pupil radius:  12
Pupil center:  (12, 3)
Pupil found
Detecting pupil
Contours:  (array([[[ 0,  0]],

       [[ 0,  7]],

       [[ 6,  7]],

       [[ 6,  5]],

       [[ 7,  4]],

       [[11,  4]],

       [[12,  5]],

       [[12,  7]],

       [[17,  7]],

       [[17,  3]],

       [[18,  2]],

       [[22,  2]],

       [[24,  4]],

       [[25,  4]],

       [[25,  0]]], dtype=int32),)
Contour area:  122.5
Approx area:  137.5
Pupil radius:  12
Pupil center:  (12, 3)
Pupil found
Processed image saved to ./processed_eloise_f0ea5a09-2377-4544-b004-5b2bbb6a257c.png
Detecting pupil
Contou